In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Loading dataset
meta = pd.read_csv('Datasets/meta.csv')

# Dropping gender column
meta = meta.drop(['gender'], axis=1)

# Filtaring dataset
meta = meta[meta['age'] >= 0]
meta = meta[meta['age'] <= 101]

# Converting into numpy array
meta = meta.values


# Making the directory structure
output_dir = 'Datasets/imdbwiki/'

if not os.path.exists(output_dir):
   os.makedirs(output_dir)

# Finally making the training and testing set
from concurrent.futures import ThreadPoolExecutor

def process_image(age_image_tuple):
    age, image = age_image_tuple
    img = cv2.imread('Datasets/' + image, 1)
    h, w = img.shape[:2]
    min_dim = min(h, w)
    top, bottom = (h - min_dim) // 2, (h + min_dim) // 2
    left, right = (w - min_dim) // 2, (w + min_dim) // 2
    img_cropped = img[top:bottom, left:right]
    path = image
    path = path[path.rindex('/')+1:]
    path = 'Datasets/imdbwiki/' + str(age) + '_' + path
    cv2.imwrite(path, img_cropped)

with ThreadPoolExecutor(max_workers=os.cpu_count()-2) as executor: # Adjust max_workers as needed
    executor.map(process_image, meta)



In [ ]:
modelFile = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [2]:
import cv2
import numpy as np
import os
from concurrent.futures import ProcessPoolExecutor

modelFile = "../res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "../deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
directory = './imdbwiki/'
faces = './imdbwiki-200/'
if not os.path.exists(faces):
   os.makedirs(faces)

def process_image(path, newpath):
    if os.path.exists(newpath):
        return
    f = cv2.imread(path, 1)
    blob = cv2.dnn.blobFromImage(cv2.resize(f, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
            (x, y, x2, y2) = box.astype("int")
            r = (x2-x)-(y2-y)
            if(r>0):
                m = (y+y2)//2
                y = m - (x2-x)//2
                y2 = m + (x2-x)//2
            elif(r<0):
                m = (x+x2)//2
                x= m - (y2-y)//2
                x2 = m + (y2-y)//2
            if(x<0 or y<0):
                continue
            extracted_face = f[y:y2, x:x2]
            height, width, channels = extracted_face.shape
            if(max(width, height)<200):
                continue
            extracted_face = cv2.resize(extracted_face, (200,200))
            cv2.imwrite(newpath, extracted_face)

print(os.cpu_count())
with ProcessPoolExecutor(max_workers=os.cpu_count()-2) as executor:
    image_files = [os.path.join(directory, filename) for filename in os.listdir(directory)]
    face_files = [os.path.join(faces, filename) for filename in os.listdir(directory)]
    executor.map(process_image, image_files,face_files)

12


KeyboardInterrupt: 

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import random
import cv2
import numpy as np
import os


In [ ]:
path_to_images = "./imdbwiki/"

# Get a list of all image filenames in the directory
image_filenames = os.listdir(path_to_images)

random_filename = random.choice(image_filenames)

    # Construct the full path to the image
full_image_path = os.path.join(path_to_images, random_filename)\

image = cv2.imread(full_image_path)

# Convert the image from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.imshow(image)
plt.show()


modelFile = "../res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "../deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

f = cv2.imread(full_image_path, 1)
blob = cv2.dnn.blobFromImage(cv2.resize(f, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
net.setInput(blob)
detections = net.forward()
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
        (x, y, x2, y2) = box.astype("int")
        r = (x2-x)-(y2-y)
        if(r > 0):
            m = (y+y2)//2
            y = m - (x2-x)//2
            y2 = m + (x2-x)//2
        elif(r < 0):
            m = (x+x2)//2
            x = m - (y2-y)//2
            x2 = m + (y2-y)//2
        if(x < 0 or y < 0):
            continue
        extracted_face = f[y:y2, x:x2]
        height, width, channels = extracted_face.shape
        print(f'The extracted face image size is: width={width}px, height={height}px, channels={channels}')
        print(f'Max = {max(width,height)}')
        plt.imshow(cv2.cvtColor(extracted_face, cv2.COLOR_BGR2RGB))
        plt.show()


In [1]:
import cv2
import numpy as np
import os
from concurrent.futures import ProcessPoolExecutor

modelFile = "../res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "../deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
directory = './imdbwiki/'
faces = './imdbwiki-200-fixed/'
multi_faces = './imdbwiki-multi/'

if not os.path.exists(faces):
   os.makedirs(faces)
if not os.path.exists(multi_faces):
   os.makedirs(multi_faces)

def process_image(path, newpath):
    if os.path.exists(newpath):
        return

    f = cv2.imread(path, 1)
    blob = cv2.dnn.blobFromImage(cv2.resize(f, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_count = 0
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            face_count += 1

    if face_count > 1:
        img_folder = os.path.join(multi_faces, os.path.splitext(os.path.basename(path))[0])
        os.makedirs(img_folder, exist_ok=True)

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
                (x, y, x2, y2) = box.astype("int")
                r = (x2-x)-(y2-y)
                if(r>0):
                    m = (y+y2)//2
                    y = m - (x2-x)//2
                    y2 = m + (x2-x)//2
                elif(r<0):
                    m = (x+x2)//2
                    x= m - (y2-y)//2
                    x2 = m + (y2-y)//2
                if(x<0 or y<0):
                    continue
                extracted_face = f[y:y2, x:x2]
                height, width, channels = extracted_face.shape
                if(max(width, height)<200):
                    continue
                extracted_face = cv2.resize(extracted_face, (200,200))
                cv2.imwrite(os.path.join(img_folder, f'face_{i}.jpg'), extracted_face)
    else:
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * \
                    np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
                (x, y, x2, y2) = box.astype("int")
                r = (x2-x)-(y2-y)
                if(r > 0):
                    m = (y+y2)//2
                    y = m - (x2-x)//2
                    y2 = m + (x2-x)//2
                elif(r < 0):
                    m = (x+x2)//2
                    x = m - (y2-y)//2
                    x2 = m + (y2-y)//2
                if(x < 0 or y < 0):
                    continue
                extracted_face = f[y:y2, x:x2]
                height, width, channels = extracted_face.shape
                if(max(width, height) < 200):
                    continue
                extracted_face = cv2.resize(extracted_face, (200, 200))
                cv2.imwrite(newpath, extracted_face)


print(os.cpu_count())
with ProcessPoolExecutor(max_workers=os.cpu_count()-1) as executor:
    image_files = [os.path.join(directory, filename) for filename in os.listdir(directory)]
    face_files = [os.path.join(faces, filename) for filename in os.listdir(directory)]
    executor.map(process_image, image_files,face_files)

12


In [1]:
import os
print(os.cpu_count())


12


In [ ]:
cv2.destroyAllWindows()